In [1]:
#import libs
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable


C:\Users\Juba\.conda\envs\py3.6\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#inti
lb_make=LabelEncoder()
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
#cd C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\NSL-KDD

In [4]:
#INput
file_name='df_testing.csv'
missing_values = ["n/a", "na", "Infinity", "NaN","nan","-","excel","?","#DIV/0!","aza"]
df = pd.read_csv("../df_training.csv", na_values = missing_values, engine='python', skipinitialspace=True)
df=df.fillna(df.median())
df = df[df['class3'] == 0]

In [5]:

class CDAE(nn.Module):
    def __init__(self):
        super(CDAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(78, 64),
            nn.ReLU(),
            nn.Linear(64, 42),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(42, 64),
            nn.ReLU(),
            nn.Linear(64, 78),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


def add_noise(data, noise_factor=0.05):
    noise = torch.randn_like(data) * noise_factor
    data_noisy = data + noise
    return data_noisy


def loss_function(W, x, recons_x, h, lambda_w=1e-4):
    mse_loss = F.mse_loss(recons_x, x)
    dh = h * (1 - h)  # Derivative of sigmoid
    w_sum = torch.sum(W**2)  # Ensure W is your weight tensor and sum of squares of weights
    # Correct calculation of contractive loss
    # Multiply the sum of squared derivatives by the sum of squared weights (scalar multiplication)
    contractive_loss = torch.sum(dh**2) * w_sum
    return mse_loss + lambda_w * contractive_loss



In [6]:

def add_noise(data, noise_factor=0.05):
    noise = torch.randn_like(data) * noise_factor
    data_noisy = data + noise
    return data_noisy

In [7]:
def train(model, data, optimizer):
    model.train()
    total_loss = 0
    data = add_noise(data)  # Add noise to data
    data = Variable(data.view(-1, 78))
    optimizer.zero_grad()
    recons = model(data)
    loss = loss_function(model.encoder[2].weight, data, recons, model.encoder[1](model.encoder[0](data)))
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    return total_loss


# Load your data here

In [8]:
# Load your data here
normal_data = df

In [9]:
df.shape

(337133, 79)

In [10]:
 
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()  # default=(0, 1)
features = normal_data.iloc[:, :-1]
features_scaled = scaler.fit_transform(features)

# Convert back to dataframe
features = pd.DataFrame(features_scaled, columns=features.columns)

# Convert the dataframe to a tensor
features = torch.tensor(features.values, dtype=torch.float32)

# Create the model
model = CDAE()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train the model on normal data
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, features, optimizer)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss))


Epoch [1/10], Loss: 433.2774
Epoch [2/10], Loss: 388.7471
Epoch [3/10], Loss: 347.3539
Epoch [4/10], Loss: 309.0034
Epoch [5/10], Loss: 273.5293
Epoch [6/10], Loss: 240.9986
Epoch [7/10], Loss: 211.2164
Epoch [8/10], Loss: 184.2187
Epoch [9/10], Loss: 159.7669
Epoch [10/10], Loss: 137.9759


In [11]:
print(features.size())


torch.Size([337133, 78])


In [12]:
'''
# Load the second CSV file and transform the data into tensors
df_adv = pd.read_csv(file_name)

adv_features = df_adv.iloc[:, :-1].values
adv_labels = df_adv.iloc[:, -1].values


# Convert to tensor
adv_features = torch.from_numpy(adv_features).float()
adv_labels = torch.from_numpy(adv_labels).long()

# Evaluate model on adversarial data
model.eval()
with torch.no_grad():
    reconstructed = model(adv_features)
df_out = pd.DataFrame(reconstructed.numpy())
df_out['42 Labels'] = adv_labels.numpy()
df_out.to_csv('test-CDAE-KDD.csv', index=False)
# Save output to csv
df_out = pd.DataFrame(reconstructed.numpy())
df_out['42 Labels'] = adv_labels.numpy()
df_out.to_csv('test-CDAE-KDD.csv', index=False)
'''

"\n# Load the second CSV file and transform the data into tensors\ndf_adv = pd.read_csv(file_name)\n\nadv_features = df_adv.iloc[:, :-1].values\nadv_labels = df_adv.iloc[:, -1].values\n\n\n# Convert to tensor\nadv_features = torch.from_numpy(adv_features).float()\nadv_labels = torch.from_numpy(adv_labels).long()\n\n# Evaluate model on adversarial data\nmodel.eval()\nwith torch.no_grad():\n    reconstructed = model(adv_features)\ndf_out = pd.DataFrame(reconstructed.numpy())\ndf_out['42 Labels'] = adv_labels.numpy()\ndf_out.to_csv('test-CDAE-KDD.csv', index=False)\n# Save output to csv\ndf_out = pd.DataFrame(reconstructed.numpy())\ndf_out['42 Labels'] = adv_labels.numpy()\ndf_out.to_csv('test-CDAE-KDD.csv', index=False)\n"

In [13]:
 cd C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\ON

C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\ON


In [14]:
def preprocess_and_save(file_name, model, scaler, output_file):
    # Load the CSV file
    df_adv = pd.read_csv(file_name)
    
    # Assuming the last column is the label
    adv_features = df_adv.iloc[:, :-1]
    adv_labels = df_adv.iloc[:, -1].values
    
    # Apply the same scaling as the original data
    adv_features_scaled = scaler.transform(adv_features)  # Use the same scaler as for training data
    
    # Convert to tensor
    adv_features_tensor = torch.tensor(adv_features_scaled, dtype=torch.float32)
    
    # Evaluate model on this data
    model.eval()
    with torch.no_grad():
        reconstructed = model(adv_features_tensor)
        
    # Convert reconstructed data back to numpy array (if needed)
    reconstructed_np = reconstructed.numpy()
    
    # Create a DataFrame from the reconstructed features
    df_out = pd.DataFrame(reconstructed_np, columns=adv_features.columns)
    
    # Append the labels column
    df_out['class3'] = adv_labels
    
    # Save to CSV
    df_out.to_csv(output_file, index=False)

# Example usage:
model.eval()  # Ensure the model is in evaluation mode
scaler = MinMaxScaler().fit(features)  # Assuming 'features' is your training features DataFrame

# Assuming you have a list of file names to process
file_names = ['FSGM-XAAE-IIoT.csv', 'BIM-XAAE-IIoT.csv', 'DF-XAAE-IIoT.csv', 'JSMA-XAAE-IIoT.csv', 'CW2-XAAE-IIoT.csv', 'CWinf-XAAE-IIoT.csv']
output_files = ['FSGM-CDAE-IIoT.csv', 'BIM-CDAE-IIoT.csv', 'DF-CDAE-IIoT.csv', 'JSMA-CDAE-IIoT.csv', 'CW2-CDAE-IIoT.csv','CWinf-CDAE-IIoT.csv']

for file_name, output_file in zip(file_names, output_files):
    preprocess_and_save(file_name, model, scaler, output_file)


In [16]:
cd ..

C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT


In [17]:
# Step 8: Apply the trained DAE to denoise the adversarial samples
adversarial_data = pd.read_csv('df_training.csv')
adversarial_features = adversarial_data.drop('class3', axis=1).values
X_adv = torch.from_numpy(adversarial_features).float()
denoised_output = model(X_adv)

# Step 9: Convert the denoised output tensor back to a NumPy array and rescale
denoised_samples = denoised_output.detach().numpy()
#denoised_samples = scaler.inverse_transform(denoised_samples)

# Step 10: Create a DataFrame with the denoised samples and assign the label 'class3'
denoised_data = pd.DataFrame(denoised_samples, columns=adversarial_data.columns[:-1])
denoised_data['class3'] = adversarial_data['class3']

denoised_data.to_csv('df_training_CDAE.csv', index=False)

In [18]:
# Step 8: Apply the trained DAE to denoise the adversarial samples
adversarial_data = pd.read_csv('df_testing.csv')
adversarial_features = adversarial_data.drop('class3', axis=1).values
X_adv = torch.from_numpy(adversarial_features).float()
denoised_output = model(X_adv)

# Step 9: Convert the denoised output tensor back to a NumPy array and rescale
denoised_samples = denoised_output.detach().numpy()
#denoised_samples = scaler.inverse_transform(denoised_samples)

# Step 10: Create a DataFrame with the denoised samples and assign the label 'class3'
denoised_data = pd.DataFrame(denoised_samples, columns=adversarial_data.columns[:-1])
denoised_data['class3'] = adversarial_data['class3']

denoised_data.to_csv('df_testing_CDAE.csv', index=False)

In [20]:
 cd C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\CDAE_DAE

C:\adversarial_analysis-master\Investigating_the_Practicality_of_Adversarial_Evasion_Attacks_on_Network_Intrusion_Detection\XAAE-IIoT\CDAE_DAE


In [21]:
torch.save(model.state_dict(), 'CDAE_model.pth')